This notebook compares the model performance between two CNN architectures - LeNet-5 and a custom CNN comically called ArchiNet - to solve multiclass image classification of the architectural style of a building from a given image.  The images dataset was retrieved from here: https://www.kaggle.com/datasets/dumitrux/architectural-styles-dataset.  After the models are trained, their performances are evaluated through visualizations.

# **Import Required Libraries**

In [ ]:
# Suppress warning messages
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Standard libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import json
import itertools
import glob
import sys
import requests
import random
import pickle
import joblib
import imageio
import PIL
from tabulate import tabulate
from pathlib import Path
from PIL import ImageFont, Image

# Machine learning libraries
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from skimage.transform import resize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D, AveragePooling2D, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# Suppress Tensorflow warnings and errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
print("Numpy version: ", np.__version__)
print("Pandas version: ", pd.__version__)
print("CV2 version: ", cv2.__version__)
print("TensorFlow version: ", tf.__version__)
print("Keras version: ", keras.__version__)

In [ ]:
# Any edits to libraries are reloaded automatically
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# **Training, Validation, and Test Sets**

In [ ]:
styles = ['Achaemenid architecture','American craftsman style','American Foursquare architecture','Ancient Egyptian architecture','Art Deco architecture',
    'Art Nouveau architecture','Baroque architecture','Bauhaus architecture','Beaux-Arts architecture','Byzantine architecture',
    'Chicago school architecture','Colonial architecture','Deconstructivism','Edwardian architecture','Georgian architecture',
    'Gothic architecture','Greek Revival architecture','International style','Novelty architecture','Palladian architecture',
    'Postmodern architecture','Queen Anne architecture','Romanesque architecture','Russian Revival architecture','Tudor Revival architecture']

In [ ]:
# Separate image dataset into features and labels (target = style)
X = []
y = []

# Set size for images to be resized to
size = (256, 256)

# Put images in X and their labels in y
for style in styles[:]:
    img_file = glob.glob(f'../input/architectural-styles-dataset/**/{style}/*.jpg', recursive = True)
    
    # Resize and grayscale each image
    for i, f in enumerate(img_file):
        img = cv2.imread(f)
        img = cv2.resize(img, size, interpolation = cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = img.astype('float32')
        img /= 255 
        X.append(img)
        y.append(style)

In [ ]:
# Cast labels to integers 
y = [styles.index(label) for label in y]

# Create training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2)

In [ ]:
# Display length of X and y
# Should be the same and reflect number of images from dataset
print(f'Length of X: {len(X)}')
print(f'Length of y: {len(y)}')

In [ ]:
# Check lengths of training, validation, and test sets
print("Length of X_train: ", len(X_train))
print("Length of y_train: ", len(y_train))
print("Length of X_valid: ", len(X_valid))
print("Length of y_valid: ", len(y_valid))
print("Length of X_test: ", len(X_test))
print("Length of y_test: ", len(y_test))

In [ ]:
# Make sets arrays for model fitting
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

type(X_train), type(y_train)

# **Build and Train Models**

## *LeNet-5 CNN*

LeNet-5 CNN architecture retrieved from:
https://towardsdatascience.com/convolutional-neural-network-champions-part-1-lenet-5-7a8d6eb98df6

In [ ]:
# Create function for LeNet-5 CNN model
def lenet5():
    model = Sequential()
    model.add(Reshape((256, 256, 1), input_shape=(256, 256), name='Reshape'))
    model.add(Conv2D(input_shape=(32, 32, 1), filters=6, kernel_size=(5, 5), strides=(1, 1), padding="same", activation='tanh', name='Conv2D_1'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid', name='AvgPool2D_1'))
    model.add(Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), padding='valid', activation='tanh', name='Conv2D_2'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid', name='AvgPool2D_2'))
    model.add(Flatten(name='Flatten'))
    model.add(Dense(120, activation='tanh', name='Dense_1'))
    model.add(Dense(84, activation='tanh', name='Dense_2'))
    model.add(Dense(25, activation='softmax', name='Dense_3'))
    return model

In [ ]:
# Assign LeNet-5
LeNet5 = lenet5()

In [ ]:
# View summary for LeNet-5
print(LeNet5.summary(line_length = 120))

In [ ]:
# Define optimizer and loss function for LeNet-5
lenet5_early_stop = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 2)
lenet5_opt = SGD(learning_rate = 0.01, momentum = 0.0, nesterov = 'False')
lenet5_loss = SparseCategoricalCrossentropy()

In [ ]:
# Compile LeNet-5 CNN model
LeNet5.compile(loss = lenet5_loss, optimizer = lenet5_opt, metrics = ['accuracy'])

In [ ]:
# Train LeNet-5 on training set and validate with validation set
lenet5_history = LeNet5.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs = 15, shuffle = True, callbacks = lenet5_early_stop)

In [ ]:
# Evaluate loss and accuracy from test set
LeNet5.evaluate(X_test, y_test)

In [ ]:
# Save model
LeNet5.save('LeNet5.h5')

In [ ]:
# Load model (for when the same kernel instance isn't in use)
LeNet5 = load_model('LeNet5.h5')

In [ ]:
# Save the history (fit) of model
with open('LeNet5TrainingHistory', 'wb') as file:
    pickle.dump(lenet5_history.history, file)

In [ ]:
# Load saved history (fit) of model (for when the same kernel instance isn't in use)
with open('LeNet5TrainingHistory', "rb") as file:
    lenet5_history = pickle.load(file)

In [ ]:
# Make prediciton on first image in test set
lenet5_pred = LeNet5.predict(X_test)
lenet5_pred[0]

In [ ]:
# Print a quick comparison of the predicted label vs actual label of first test sample
print(f'Predicted encoded label of first test sample: {np.argmax(lenet5_pred[0])}')
print(f'Actual encoded label of first test sample: {y_test[0]}')

In [ ]:
# Max prediction of each label for each image
lenet5_y_pred = np.argmax(lenet5_pred, axis = 1)

In [ ]:
# Display first 25 images from test set and the predicted label
fig, ax = plt.subplots(ncols = 5, nrows = 5, figsize = (12, 12))
ax = ax.flatten()

for i, img in enumerate(X_test[:25]):
    ax[i].axis('off')
    ax[i].imshow(img, cmap = "binary", interpolation = "nearest")
    ax[i].set_title(styles[lenet5_y_pred[i]], fontsize = 9)

fig.suptitle('LeNet5 Test Dataset Predictions', fontsize = 16)

## *Custom CNN - ArchiNet*

In [ ]:
def archinet():
    model = Sequential()
    model.add(Reshape((256, 256, 1), input_shape=(256, 256), name='Reshape'))
    model.add(Conv2D(input_shape=(256, 256, 3), filters=6, kernel_size=(3, 3), padding='same', activation='relu', name='Conv2D_1'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='MaxPool2D_1'))
    model.add(Conv2D(filters=24, kernel_size=(3, 3), padding='same', activation='relu', name='Conv2D_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='MaxPool2D_2'))
    model.add(Conv2D(filters=48, kernel_size=(3, 3), padding='same', activation='relu', name='Conv2D_3'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='MaxPool2D_3'))
    model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='same', activation='relu', name='Conv2D_4'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='MaxPool2D_4'))
    model.add(Flatten(name='Flatten'))
    model.add(Dense(125, activation='relu', name='Dense_1'))
    model.add(Dense(75, activation='relu', name='Dense_2'))
    model.add(Dense(25, activation='softmax', name='Dense_3'))
    return model

In [ ]:
# Assign custom CNN
ArchiNet = archinet()

In [ ]:
# View summary for custom CNN
print(ArchiNet.summary())

In [ ]:
# Define callback (early stopping), optimizer and loss function for Custom CNN
archinet_early_stop = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 5)
archinet_opt = Adam(learning_rate = 0.001)
archinet_loss = SparseCategoricalCrossentropy()

In [ ]:
# Compile Custom CNN model
ArchiNet.compile(loss = archinet_loss, optimizer = archinet_opt, metrics = ['accuracy'])

In [ ]:
# Train Custom CNN on training set and validate with validation set
archinet_history = ArchiNet.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs = 30, shuffle = True, callbacks = archinet_early_stop)

In [ ]:
# Evaluate loss and accuracy from test set
ArchiNet.evaluate(X_test, y_test)

In [ ]:
# Save model
ArchiNet.save('ArchiNet.h5')

In [ ]:
# Load model (for when the same kernel instance isn't in use)
ArchiNet = load_model('ArchiNet.h5')

In [ ]:
# Save the history (fit) of model
with open('ArchiNetTrainingHistory', 'wb') as file:
    pickle.dump(archinet_history.history, file)

In [ ]:
# Load saved history (fit) of model (for when the same kernel instance isn't in use)
with open('ArchiNetTrainingHistory', "rb") as file:
    archinet_history = pickle.load(file)

In [ ]:
# Make prediciton on first image in test set
archinet_pred = ArchiNet.predict(X_test)
archinet_pred[0]

In [ ]:
# Print a quick comparison of the predicted label vs actual label of first test sample
print(f'Predicted encoded label of first test sample: {np.argmax(archinet_pred[0])}')
print(f'Actual encoded label of first test sample: {y_test[0]}')

In [ ]:
# Max prediction of each label for each image
archinet_y_pred = np.argmax(archinet_pred, axis = 1)

In [ ]:
# Display first 25 images from test set and the predicted label
fig, ax = plt.subplots(ncols = 5, nrows = 5, figsize = (12, 12))
ax = ax.flatten()

for i, img in enumerate(X_test[:25]):
    ax[i].axis('off')
    ax[i].imshow(img, cmap = "binary", interpolation = "nearest")
    ax[i].set_title(styles[archinet_y_pred[i]], fontsize = 9)

fig.suptitle('ArchiNet Test Dataset Predictions', fontsize = 16)

# **Visualize Model Performance**

## *LeNet-5 CNN*

In [ ]:
# Print performance evaluation metrics for model
print(f"LeNet-5 Accuracy Score: {accuracy_score(lenet5_y_pred, y_test)}")
print(f"LeNet-5 Precision Score: {precision_score(lenet5_y_pred, y_test, average = 'weighted')}")
print(f"LeNet-5 Recall Score: {recall_score(lenet5_y_pred, y_test, average = 'weighted')}")
print(f"LeNet-5 F1 Score: {f1_score(lenet5_y_pred, y_test, average = 'weighted')}")

In [ ]:
# Plot LeNet-5 accuracy and loss for training and validation sets from model history
plt.subplots(figsize = (10, 5))
plt.subplots_adjust(wspace = 0.5)

# Accuracy plot
plt.subplot(1, 2, 1)
plt.grid('on')
plt.plot(lenet5_history['accuracy'], color = "green")
plt.plot(lenet5_history['val_accuracy'], color = "blue")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('LeNet-5 Accuracy', fontsize = 15, fontweight = "bold")
plt.legend(['Training', 'Validation'], loc = 'upper left')

# Loss plot
plt.subplot(1, 2, 2)
plt.grid('on')
plt.plot(lenet5_history['loss'], color = "red")
plt.plot(lenet5_history['val_loss'], color = "black")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('LeNet-5 Loss', fontsize = 15, fontweight = "bold")
plt.legend(['Training', 'Validation'], loc = 'upper left')

# Show plots
plt.show()

In [ ]:
# Create y_true variable to represent the actual labels from y_test
# Will be used across all confusion matrices
y_true = y_test

In [ ]:
# Create confusion matrix from model predictions
lenet5_conf_mtx = confusion_matrix(y_true, lenet5_y_pred)

In [ ]:
# Display confusion matrix
fig = plt.figure(figsize = (16, 14))
ax = plt.subplot()

# Set heatmap for confusion matrix
sns.heatmap(lenet5_conf_mtx, annot = True, ax = ax, cmap = "icefire", fmt = 'g')

# Set axes labels
ax.xaxis.set_label_position('bottom')
ax.set_xlabel('Predicted', fontsize = 20)
ax.set_ylabel('True', fontsize = 20)

# Set axes ticks
ax.xaxis.tick_bottom()
ax.xaxis.set_ticklabels(styles, fontsize = 10)
ax.yaxis.set_ticklabels(styles, fontsize = 10)
plt.xticks(rotation = 90)
plt.yticks(rotation = 0)

plt.title('LeNet-5 Confusion Matrix', fontsize = 16)

In [ ]:
# Create list to append X_test images that were incorrectly predicted
lenet5_wrong = []

for i in range(len(lenet5_y_pred)):
    if lenet5_y_pred[i] != y_true[i]:
        lenet5_wrong.append(X_test[i])

In [ ]:
# Display 25 wrong predictions
fig, ax = plt.subplots(ncols = 5, nrows = 5, figsize = (12, 12))
ax = ax.flatten()

# Set random starting point and ending point
rand_start = random.randint(0, 300)
rand_end = rand_start + 25

for i, img in enumerate(lenet5_wrong[rand_start:rand_end]):
    ax[i].axis('off')
    ax[i].imshow(img, cmap = "binary", interpolation = "nearest")
    ax[i].set_title(styles[lenet5_y_pred[i]], fontsize = 9)

fig.suptitle('LeNet-5 Wrong Predictions', fontsize = 16)

## *Custom CNN - ArchiNet*

In [ ]:
# Print performance evaluation metrics for model
print(f"ArchiNet Accuracy Score: {accuracy_score(archinet_y_pred, y_test)}")
print(f"ArchiNet Precision Score: {precision_score(archinet_y_pred, y_test, average = 'weighted')}")
print(f"ArchiNet Recall Score: {recall_score(archinet_y_pred, y_test, average = 'weighted')}")
print(f"ArchiNet F1 Score: {f1_score(archinet_y_pred, y_test, average = 'weighted')}")

In [ ]:
# Plot accuracy and loss for training and validation sets from model history
plt.subplots(figsize = (10, 5))
plt.subplots_adjust(wspace = 0.5)

# Accuracy plot
plt.subplot(1, 2, 1)
plt.grid('on')
plt.plot(archinet_history['accuracy'], color = "green")
plt.plot(archinet_history['val_accuracy'], color = "blue")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('ArchiNet Accuracy', fontsize = 15, fontweight = "bold")
plt.legend(['Training', 'Validation'], loc = 'upper left')

# Loss plot
plt.subplot(1, 2, 2)
plt.grid('on')
plt.plot(archinet_history['loss'], color = "red")
plt.plot(archinet_history['val_loss'], color = "black")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('ArchiNet Loss', fontsize = 15, fontweight = "bold")
plt.legend(['Training', 'Validation'], loc = 'upper left')

# Show plots
plt.show()

In [ ]:
# Create confusion matrix from model predictions
archinet_conf_mtx = confusion_matrix(y_true, archinet_y_pred)

In [ ]:
# Display confusion matrix
fig = plt.figure(figsize = (16, 14))
ax = plt.subplot()

# Set heatmap for confusion matrix
sns.heatmap(archinet_conf_mtx, annot = True, ax = ax, cmap = "mako", fmt = 'g')

# Set axes labels
ax.xaxis.set_label_position('bottom')
ax.set_xlabel('Predicted', fontsize = 20)
ax.set_ylabel('True', fontsize = 20)

# Set axes ticks
ax.xaxis.tick_bottom()
ax.xaxis.set_ticklabels(styles, fontsize = 10)
ax.yaxis.set_ticklabels(styles, fontsize = 10)
plt.xticks(rotation = 90)
plt.yticks(rotation = 0)

plt.title('ArchiNet Confusion Matrix', fontsize = 16)

In [ ]:
# Create list to append X_test images that were incorrectly predicted
archinet_wrong = []

for i in range(len(archinet_y_pred)):
    if archinet_y_pred[i] != y_true[i]:
        archinet_wrong.append(X_test[i])

In [ ]:
# Display 25 wrong predictions
fig, ax = plt.subplots(ncols = 5, nrows = 5, figsize = (12, 12))
ax = ax.flatten()

# Set random starting point and ending point
rand_start = random.randint(0, 300)
rand_end = rand_start + 25

for i, img in enumerate(archinet_wrong[rand_start:rand_end]):
    ax[i].axis('off')
    ax[i].imshow(img, cmap = "binary", interpolation = "nearest")
    ax[i].set_title(styles[archinet_y_pred[i]], fontsize = 9)

fig.suptitle('ArchiNet Wrong Predictions', fontsize = 16)

In [ ]:
# Print summary of performance metrics comparing CNNs
print(tabulate([['Lenet-5', 'Accuracy', '{:,.2%}'.format(accuracy_score(lenet5_y_pred, y_test))], 
                ['', 'Precision', '{:,.2%}'.format(precision_score(lenet5_y_pred, y_test, average = 'weighted'))],
                ['', 'Recall', '{:,.2%}'.format(recall_score(lenet5_y_pred, y_test, average = 'weighted'))],
                ['', 'F1', '{:,.2%}'.format(f1_score(lenet5_y_pred, y_test, average = 'weighted'))],
                ['', '', '', ''],
                ['Archinet', 'Accuracy', '{:,.2%}'.format(accuracy_score(archinet_y_pred, y_test))], 
                ['', 'Precision', '{:,.2%}'.format(precision_score(archinet_y_pred, y_test, average = 'weighted'))],
                ['', 'Recall', '{:,.2%}'.format(recall_score(archinet_y_pred, y_test, average = 'weighted'))],
                ['', 'F1', '{:,.2%}'.format(f1_score(archinet_y_pred, y_test, average = 'weighted'))]],
               headers=['Model', 'Metric', 'Performance']))